# Notebook Setup

In [ ]:
import os
from dotenv import load_dotenv

import ep_parse.core as core
import ep_parse.case_data as d
import ep_parse.utils as u
import ep_parse.nb_utils.signal_display as sd

load_dotenv()
u.configure_logging()

%load_ext autoreload
%autoreload 2

case_id = "CASE002"

## Initialize Case and view exported signal coverage

In [ ]:
# create a meta file for storing case properties
d.create_case_meta(case_id)

# view time intervals for which signal data is present (per channel)
core.plot_raw_data_coverage(case_id)

# Clean Files exported from EPMED
Warning - Destructive Operations - be sure to backup your raw export files

In [ ]:
import ep_parse.epmed.file_cleanup as cln

# CASE_BIN_DIRECTORY = f"data/{case_id}/epmed"  # custom path
CASE_BIN_DIRECTORY = d.case_file_path(case_id, d.DataSource.EPMED)  # standard path

In [ ]:
# Rename BIN files to remove inconsistent spaces and capitalization
cln.rename_BINS(CASE_BIN_DIRECTORY, False)

In [ ]:
# Remove redundant Page files (just reduce storage size for these raw files)
cln.remove_extra_page_files(CASE_BIN_DIRECTORY, qa_check=False)

In [ ]:
# Remove data TXT files
cln.remove_TXT_datafiles(CASE_BIN_DIRECTORY, True)

In [ ]:
# Remove name row from session text files ... if applicable
cln.remove_pii(CASE_BIN_DIRECTORY, case_id)

In [ ]:
### Remove duplicate overlapping exports
to_remove = cln.check_partial_export(CASE_BIN_DIRECTORY, assert_if=False)
for f in to_remove:
    os.remove(os.path.join(CASE_BIN_DIRECTORY, f))

# Write Formatted Data Files

## Write Signal Data file

Clean and transform signal data that was exported from the research center's epsystem and write it to a `signals.h5` file.<br>
**Prereqs**
- Exported signal data conforms to the standardized layout (see README) and is located in a folder named by the case_id (e.g. folder name is `CASE001`) in the `os.environ.get("data_filepath")` directory.


**Output**
- An h5 file (`{case_id}_signals.h5`) file in the case's root directory.

In [ ]:
to_import = {"events", "signals"}  # subset of {"signals", "events"} specifying which data to parse (events = bookmarks log file)
### End Configs

core.format_and_store_case_export(case_id, only=to_import, compress=False)
d.store_pps_in_meta(case_id)

## Plot signals

### By RF

In [ ]:
rfs = "5,10"  # format is "1,2,3,4-6,12"

rf_configs = {
    "width": 40,  # set plot width
    "height": 10,  # adjust plot height
    "prepend_seconds": 4,
    "append_seconds": 4,
    "y_scale": 2,  # adjust the spacing of traces within a plot
    "suppress_channels": [],  # list channels which should not appear in the plot
    "prepend_seconds": 6,
    "append_seconds": 2,
}

_ = sd.simple_signals_plot(case_id, "RF", rfs, rf_configs)

### By Time

In [ ]:
start_times = "15:45:10|after|20|8"  # "11:20:01, 11:22:12|before|12|2, 11:33:13|after|20|4"

time_configs = {
    "width": 40,  # set plot width
    "height": 10,  # adjust plot height
    "prepend_seconds": 4,
    "append_seconds": 4,
    "y_scale": 2,  # adjust the spacing of traces within a plot
    "suppress_channels": [],  # list channels which should not appear in the plot
    "prepend_seconds": 8,
    "append_seconds": 0,
}

_ = sd.simple_signals_plot(case_id, "TIME", start_times, time_configs)

## Generate bookmarks file for Medtronic catheter
Occassionally catheters will be used that records RF times in a single file per RF format.  These cells are used to convert that data export structure into a standard bookmarks.txt file

### Store md catheter time offset for the case

In [ ]:
epmed_time = "12:00:00.000"  # "12:00:00.000" format
md_cath_time = "12:00:00.000"  # time in md system corresponding to epmed_time above
FORCE = False  # override the value from the case meta file

#### End Configs

offset = core.store_md_cath_offset(case_id, epmed_time, md_cath_time, force=FORCE)

### Create bookmarks file from modified Lesion file

In [ ]:
core.md_catheter_logs_as_bookmark_file(case_id)
core.load_events(case_id, outputs=['ep_system_events'])

# Bandwidth Filtering

In [ ]:
import ep_parse.bandwidth_filter as bwf
import ep_parse.signal_nav as sn

# Assess a random chunk at RF 2 to see if there is bandwidth noise
# omni_sig = sn.lookup_by_rf(bookmark_df, 2, 2, case_id=case_id, channels=case_meta["channels"])
signals_df = sn.lookup_by_time("15:48:30", case_id=case_id, channels=["I", "CS_1-2"])
bwf.plot_fourier_xform(signals_df, ["I", "CS_1-2"])

In [ ]:
# Based on the plots above, determine the noisy channels and notch frequency 
# ['I','II','III','aVL','aVR','aVF','V1', 'CS_1-2', 'CS_3-4', 'CS_5-6', 'CS_7-8', 'CS_9-10', 'ABLd', 'ABLp']
noisy_channels = ['I']
notch_frequency = 60.0  # Frequency to be removed from signal (Hz) - Determine this with Fourier logic above (peaks)
quality_factor = 30.0

# plot the channels after denoising to validate the process
for ch in noisy_channels:
    bwf.plot_denoised(signals_df[ch], notch_frequency, quality_factor, title_prefix=ch)

In [ ]:
# Write the new BIN files to a local directory
_bins = set(bwf.write_denoised_bins(case_id, noisy_channels, notch_frequency, quality_factor))

In [ ]:
# Replace the noisy BIN files with the denoised files (DESTRUCTIVE!!!)
import shutil
for f in _bins:
    dest = shutil.move(f"local/{f}", d.case_file_path(case_id, d.DataSource.EPMED))